In [26]:
import lyricwikia
import pandas as pd

df = pd.read_csv("pop_playlist_project.csv")
df.columns = ['Spotify_URI','TrackName','ArtistName','AlbumName','DiscNum','TrackNum','TrackDur','AddedBy','AddedAt']
lyrics = ""

for i in range(int(len(df)/120)):
    try:
        lyrics_temp = lyricwikia.get_lyrics(df.ArtistName[i], df.TrackName[i])
        lyrics=lyrics+lyrics_temp
    except:
        continue





In [27]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

In [28]:
raw_text = lyrics.lower()
chars = sorted(list(set(raw_text)))
char_to_int = dict((c,i) for i, c in enumerate(chars))

n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ",n_vocab)

Total Characters:  8513
Total Vocab:  33


In [33]:
seq_length = int(len(df)/120)
dataX = []
dataY = []
for i in range(0,n_chars - seq_length,1):
    seq_in = raw_text[i:i +seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ",n_patterns)
print(len(dataX))
print(len(dataY))

Total Patterns:  8506
8506
8506


In [34]:
X = numpy.reshape(dataX,(n_patterns,seq_length,1))
print(X.shape)
X = X/float(n_vocab)
y=np_utils.to_categorical(dataY)
print(X.shape[1])
print(X.shape[2])
print(y.shape[1])
print(X.shape)
print(y.shape)


(8506, 7, 1)
7
1
33
(8506, 7, 1)
(8506, 33)


In [35]:
model = Sequential()
model.add(LSTM(100,input_shape=(X.shape[1],X.shape[2]),return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [37]:
model.fit(X,y,epochs=10,batch_size=150)

Epoch 1/10
8506/8506 [==============================] - 5s 539us/step - loss: 3.0091
Epoch 2/10
8506/8506 [==============================] - 5s 570us/step - loss: 3.0052
Epoch 3/10
8506/8506 [==============================] - 5s 577us/step - loss: 2.9996
Epoch 4/10
8506/8506 [==============================] - 5s 562us/step - loss: 2.9946
Epoch 5/10
8506/8506 [==============================] - 5s 573us/step - loss: 2.9771
Epoch 6/10
8506/8506 [==============================] - 4s 526us/step - loss: 2.9499
Epoch 7/10
8506/8506 [==============================] - 5s 545us/step - loss: 2.9037
Epoch 8/10
8506/8506 [==============================] - 4s 483us/step - loss: 2.8569
Epoch 9/10
8506/8506 [==============================] - 5s 585us/step - loss: 2.8187
Epoch 10/10
8506/8506 [==============================] - 5s 605us/step - loss: 2.7825


In [38]:
int_to_char = dict((i,c) for i,c in enumerate(chars))
initial_text="this is"
pattern=[char_to_int[char] for char in initial_text]
pattern=pattern[0:seq_length]

#initial_text will be our user input, here it must be about 7 chars in length(including spaces)"
text_gen=""

for i in range(1000):
    x = numpy.reshape(pattern,(1,len(pattern),1))
    x = x/float(n_vocab)
    prediction = model.predict(x,verbose=0)
    prediction = prediction[0]
    index = numpy.random.choice(len(prediction), p = prediction)
    result = int_to_char[index]
    text_gen = text_gen+result
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print(text_gen)
    


[26, 15, 16, 25, 1, 16, 25]
[[[0.78787879]
  [0.45454545]
  [0.48484848]
  [0.75757576]
  [0.03030303]
  [0.48484848]
  [0.75757576]]]
[[[0.45454545]
  [0.48484848]
  [0.75757576]
  [0.03030303]
  [0.48484848]
  [0.75757576]
  [0.66666667]]]
[[[0.48484848]
  [0.75757576]
  [0.03030303]
  [0.48484848]
  [0.75757576]
  [0.66666667]
  [0.81818182]]]
[[[0.75757576]
  [0.03030303]
  [0.48484848]
  [0.75757576]
  [0.66666667]
  [0.81818182]
  [0.03030303]]]
[[[0.03030303]
  [0.48484848]
  [0.75757576]
  [0.66666667]
  [0.81818182]
  [0.03030303]
  [0.03030303]]]
[[[0.48484848]
  [0.75757576]
  [0.66666667]
  [0.81818182]
  [0.03030303]
  [0.03030303]
  [0.87878788]]]
[[[0.75757576]
  [0.66666667]
  [0.81818182]
  [0.03030303]
  [0.03030303]
  [0.87878788]
  [0.24242424]]]
[[[0.66666667]
  [0.81818182]
  [0.03030303]
  [0.03030303]
  [0.87878788]
  [0.24242424]
  [0.63636364]]]
[[[0.81818182]
  [0.03030303]
  [0.03030303]
  [0.87878788]
  [0.24242424]
  [0.63636364]
  [0.36363636]]]
[[[0.0303

[[[0.66666667]
  [0.03030303]
  [0.27272727]
  [0.78787879]
  [0.        ]
  [0.33333333]
  [0.48484848]]]
[[[0.03030303]
  [0.27272727]
  [0.78787879]
  [0.        ]
  [0.33333333]
  [0.48484848]
  [0.33333333]]]
[[[0.27272727]
  [0.78787879]
  [0.        ]
  [0.33333333]
  [0.48484848]
  [0.33333333]
  [0.60606061]]]
[[[0.78787879]
  [0.        ]
  [0.33333333]
  [0.48484848]
  [0.33333333]
  [0.60606061]
  [0.81818182]]]
[[[0.        ]
  [0.33333333]
  [0.48484848]
  [0.33333333]
  [0.60606061]
  [0.81818182]
  [0.03030303]]]
[[[0.33333333]
  [0.48484848]
  [0.33333333]
  [0.60606061]
  [0.81818182]
  [0.03030303]
  [0.39393939]]]
[[[0.48484848]
  [0.33333333]
  [0.60606061]
  [0.81818182]
  [0.03030303]
  [0.39393939]
  [0.39393939]]]
[[[0.33333333]
  [0.60606061]
  [0.81818182]
  [0.03030303]
  [0.39393939]
  [0.39393939]
  [0.84848485]]]
[[[0.60606061]
  [0.81818182]
  [0.03030303]
  [0.39393939]
  [0.39393939]
  [0.84848485]
  [0.36363636]]]
[[[0.81818182]
  [0.03030303]
  [0.39

[[[0.45454545]
  [0.66666667]
  [0.03030303]
  [0.57575758]
  [0.45454545]
  [0.48484848]
  [0.03030303]]]
[[[0.66666667]
  [0.03030303]
  [0.57575758]
  [0.45454545]
  [0.48484848]
  [0.03030303]
  [0.78787879]]]
[[[0.03030303]
  [0.57575758]
  [0.45454545]
  [0.48484848]
  [0.03030303]
  [0.78787879]
  [0.24242424]]]
[[[0.57575758]
  [0.45454545]
  [0.48484848]
  [0.03030303]
  [0.78787879]
  [0.24242424]
  [0.87878788]]]
[[[0.45454545]
  [0.48484848]
  [0.03030303]
  [0.78787879]
  [0.24242424]
  [0.87878788]
  [0.84848485]]]
[[[0.48484848]
  [0.03030303]
  [0.78787879]
  [0.24242424]
  [0.87878788]
  [0.84848485]
  [0.03030303]]]
[[[0.03030303]
  [0.78787879]
  [0.24242424]
  [0.87878788]
  [0.84848485]
  [0.03030303]
  [0.48484848]]]
[[[0.78787879]
  [0.24242424]
  [0.87878788]
  [0.84848485]
  [0.03030303]
  [0.48484848]
  [0.45454545]]]
[[[0.24242424]
  [0.87878788]
  [0.84848485]
  [0.03030303]
  [0.48484848]
  [0.45454545]
  [0.03030303]]]
[[[0.87878788]
  [0.84848485]
  [0.03

[[[0.75757576]
  [0.03030303]
  [0.54545455]
  [0.45454545]
  [0.48484848]
  [0.03030303]
  [0.45454545]]]
[[[0.03030303]
  [0.54545455]
  [0.45454545]
  [0.48484848]
  [0.03030303]
  [0.45454545]
  [0.45454545]]]
[[[0.54545455]
  [0.45454545]
  [0.48484848]
  [0.03030303]
  [0.45454545]
  [0.45454545]
  [0.36363636]]]
[[[0.45454545]
  [0.48484848]
  [0.03030303]
  [0.45454545]
  [0.45454545]
  [0.36363636]
  [0.24242424]]]
[[[0.48484848]
  [0.03030303]
  [0.45454545]
  [0.45454545]
  [0.36363636]
  [0.24242424]
  [0.33333333]]]
[[[0.03030303]
  [0.45454545]
  [0.45454545]
  [0.36363636]
  [0.24242424]
  [0.33333333]
  [0.36363636]]]
[[[0.45454545]
  [0.45454545]
  [0.36363636]
  [0.24242424]
  [0.33333333]
  [0.36363636]
  [0.78787879]]]
[[[0.45454545]
  [0.36363636]
  [0.24242424]
  [0.33333333]
  [0.36363636]
  [0.78787879]
  [0.24242424]]]
[[[0.36363636]
  [0.24242424]
  [0.33333333]
  [0.36363636]
  [0.78787879]
  [0.24242424]
  [0.03030303]]]
[[[0.24242424]
  [0.33333333]
  [0.36

[[[0.        ]
  [0.18181818]
  [0.87878788]
  [0.36363636]
  [0.75757576]
  [0.45454545]
  [0.33333333]]]
[[[0.18181818]
  [0.87878788]
  [0.36363636]
  [0.75757576]
  [0.45454545]
  [0.33333333]
  [0.39393939]]]
[[[0.87878788]
  [0.36363636]
  [0.75757576]
  [0.45454545]
  [0.33333333]
  [0.39393939]
  [0.63636364]]]
[[[0.36363636]
  [0.75757576]
  [0.45454545]
  [0.33333333]
  [0.39393939]
  [0.63636364]
  [0.03030303]]]
[[[0.75757576]
  [0.45454545]
  [0.33333333]
  [0.39393939]
  [0.63636364]
  [0.03030303]
  [0.75757576]]]
[[[0.45454545]
  [0.33333333]
  [0.39393939]
  [0.63636364]
  [0.03030303]
  [0.75757576]
  [0.48484848]]]
[[[0.33333333]
  [0.39393939]
  [0.63636364]
  [0.03030303]
  [0.75757576]
  [0.48484848]
  [0.36363636]]]
[[[0.39393939]
  [0.63636364]
  [0.03030303]
  [0.75757576]
  [0.48484848]
  [0.36363636]
  [0.3030303 ]]]
[[[0.63636364]
  [0.03030303]
  [0.75757576]
  [0.48484848]
  [0.36363636]
  [0.3030303 ]
  [0.24242424]]]
[[[0.03030303]
  [0.75757576]
  [0.48

...

[[[0.03030303]
  [0.27272727]
  [0.48484848]
  [0.81818182]
  [0.18181818]
  [0.33333333]
  [0.48484848]]]
[[[0.27272727]
  [0.48484848]
  [0.81818182]
  [0.18181818]
  [0.33333333]
  [0.48484848]
  [0.66666667]]]
[[[0.48484848]
  [0.81818182]
  [0.18181818]
  [0.33333333]
  [0.48484848]
  [0.66666667]
  [0.57575758]]]
[[[0.81818182]
  [0.18181818]
  [0.33333333]
  [0.48484848]
  [0.66666667]
  [0.57575758]
  [0.03030303]]]
[[[0.18181818]
  [0.33333333]
  [0.48484848]
  [0.66666667]
  [0.57575758]
  [0.03030303]
  [0.78787879]]]
[[[0.33333333]
  [0.48484848]
  [0.66666667]
  [0.57575758]
  [0.03030303]
  [0.78787879]
  [0.75757576]]]
[[[0.48484848]
  [0.66666667]
  [0.57575758]
  [0.03030303]
  [0.78787879]
  [0.75757576]
  [0.81818182]]]
[[[0.66666667]
  [0.57575758]
  [0.03030303]
  [0.78787879]
  [0.75757576]
  [0.81818182]
  [0.03030303]]]
[[[0.57575758]
  [0.03030303]
  [0.78787879]
  [0.75757576]
  [0.81818182]
  [0.03030303]
  [0.87878788]]]
[[[0.03030303]
  [0.78787879]
  [0.75

[[[0.66666667]
  [0.36363636]
  [0.63636364]
  [0.36363636]
  [0.03030303]
  [0.78787879]
  [0.03030303]]]
[[[0.36363636]
  [0.63636364]
  [0.36363636]
  [0.03030303]
  [0.78787879]
  [0.03030303]
  [0.57575758]]]
[[[0.63636364]
  [0.36363636]
  [0.03030303]
  [0.78787879]
  [0.03030303]
  [0.57575758]
  [0.66666667]]]
[[[0.36363636]
  [0.03030303]
  [0.78787879]
  [0.03030303]
  [0.57575758]
  [0.66666667]
  [0.03030303]]]
[[[0.03030303]
  [0.78787879]
  [0.03030303]
  [0.57575758]
  [0.66666667]
  [0.03030303]
  [0.93939394]]]
[[[0.78787879]
  [0.03030303]
  [0.57575758]
  [0.66666667]
  [0.03030303]
  [0.93939394]
  [0.75757576]]]
[[[0.03030303]
  [0.57575758]
  [0.66666667]
  [0.03030303]
  [0.93939394]
  [0.75757576]
  [0.36363636]]]
[[[0.57575758]
  [0.66666667]
  [0.03030303]
  [0.93939394]
  [0.75757576]
  [0.36363636]
  [0.03030303]]]
[[[0.66666667]
  [0.03030303]
  [0.93939394]
  [0.75757576]
  [0.36363636]
  [0.03030303]
  [0.03030303]]]
[[[0.03030303]
  [0.93939394]
  [0.75

[[[0.57575758]
  [0.39393939]
  [0.54545455]
  [0.24242424]
  [0.03030303]
  [0.60606061]
  [0.24242424]]]
[[[0.39393939]
  [0.54545455]
  [0.24242424]
  [0.03030303]
  [0.60606061]
  [0.24242424]
  [0.63636364]]]
[[[0.54545455]
  [0.24242424]
  [0.03030303]
  [0.60606061]
  [0.24242424]
  [0.63636364]
  [0.24242424]]]
[[[0.24242424]
  [0.03030303]
  [0.60606061]
  [0.24242424]
  [0.63636364]
  [0.24242424]
  [0.        ]]]
[[[0.03030303]
  [0.60606061]
  [0.24242424]
  [0.63636364]
  [0.24242424]
  [0.        ]
  [0.60606061]]]
[[[0.60606061]
  [0.24242424]
  [0.63636364]
  [0.24242424]
  [0.        ]
  [0.60606061]
  [0.66666667]]]
[[[0.24242424]
  [0.63636364]
  [0.24242424]
  [0.        ]
  [0.60606061]
  [0.66666667]
  [0.81818182]]]
[[[0.63636364]
  [0.24242424]
  [0.        ]
  [0.60606061]
  [0.66666667]
  [0.81818182]
  [0.36363636]]]
[[[0.24242424]
  [0.        ]
  [0.60606061]
  [0.66666667]
  [0.81818182]
  [0.36363636]
  [0.        ]]]
[[[0.        ]
  [0.60606061]
  [0.66

[[[0.03030303]
  [0.78787879]
  [0.36363636]
  [0.39393939]
  [0.42424242]
  [0.57575758]
  [0.03030303]]]
[[[0.78787879]
  [0.36363636]
  [0.39393939]
  [0.42424242]
  [0.57575758]
  [0.03030303]
  [0.03030303]]]
[[[0.36363636]
  [0.39393939]
  [0.42424242]
  [0.57575758]
  [0.03030303]
  [0.03030303]
  [0.36363636]]]
[[[0.39393939]
  [0.42424242]
  [0.57575758]
  [0.03030303]
  [0.03030303]
  [0.36363636]
  [0.66666667]]]
[[[0.42424242]
  [0.57575758]
  [0.03030303]
  [0.03030303]
  [0.36363636]
  [0.66666667]
  [0.66666667]]]
[[[0.57575758]
  [0.03030303]
  [0.03030303]
  [0.36363636]
  [0.66666667]
  [0.66666667]
  [0.03030303]]]
[[[0.03030303]
  [0.03030303]
  [0.36363636]
  [0.66666667]
  [0.66666667]
  [0.03030303]
  [0.24242424]]]
[[[0.03030303]
  [0.36363636]
  [0.66666667]
  [0.66666667]
  [0.03030303]
  [0.24242424]
  [0.72727273]]]
[[[0.36363636]
  [0.66666667]
  [0.66666667]
  [0.03030303]
  [0.24242424]
  [0.72727273]
  [0.45454545]]]
[[[0.66666667]
  [0.66666667]
  [0.03

[[[0.78787879]
  [0.75757576]
  [0.03030303]
  [0.27272727]
  [0.63636364]
  [0.03030303]
  [0.        ]]]
[[[0.75757576]
  [0.03030303]
  [0.27272727]
  [0.63636364]
  [0.03030303]
  [0.        ]
  [0.24242424]]]
[[[0.03030303]
  [0.27272727]
  [0.63636364]
  [0.03030303]
  [0.        ]
  [0.24242424]
  [0.24242424]]]
[[[0.27272727]
  [0.63636364]
  [0.03030303]
  [0.        ]
  [0.24242424]
  [0.24242424]
  [0.48484848]]]
[[[0.63636364]
  [0.03030303]
  [0.        ]
  [0.24242424]
  [0.24242424]
  [0.48484848]
  [0.        ]]]
[[[0.03030303]
  [0.        ]
  [0.24242424]
  [0.24242424]
  [0.48484848]
  [0.        ]
  [0.24242424]]]
[[[0.        ]
  [0.24242424]
  [0.24242424]
  [0.48484848]
  [0.        ]
  [0.24242424]
  [0.93939394]]]
[[[0.24242424]
  [0.24242424]
  [0.48484848]
  [0.        ]
  [0.24242424]
  [0.93939394]
  [0.45454545]]]
[[[0.24242424]
  [0.48484848]
  [0.        ]
  [0.24242424]
  [0.93939394]
  [0.45454545]
  [0.60606061]]]
[[[0.48484848]
  [0.        ]
  [0.24